# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!curl -k -o estilo.jpg https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!curl -k -o contenido.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output


In [1]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\contenido.jpg")
style_reference_image_path = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\estilo.jpg")
result_prefix = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\output")
iterations = 100

In [2]:
# Para el ejercicio 9

base_image_path = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\contenido_ejercicio_9.jpg")
style_reference_image_path = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\estilo_ejercicio_9.jpg")
result_prefix = Path("C:\\Users\\WonderBrands\\Desktop\\ITBA\\TP final\\output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:
Dichos parámetros se utilizan para ponderar las diferentes componentes de la función de pérdida durante la optimización de la imagen generada:

- total_variation_weight, se utiliza para la regularización de la variación total en la imagen generada. Ayuda a mantener la imagen generada suave y continua, reduciendo el ruido y las anomalías. Un valor más alto dará como resultado una imagen más suave ya que penalizará las transiciones abruptas entre píxeles en la imagen generada.

- style_weight, controla cuánto se prioriza la correspondencia del estilo de la imagen generada con el de la imagen de estilo. Un valor más alto dará como resultado una imagen que se asemeja más estrechamente al estilo de la imagen de estilo.

- content_weight, controla cuánto se prioriza la correspondencia del contenido de la imagen generada con el de la imagen de contenido. Un valor más alto dará como resultado una imagen que conserva más fuertemente el contenido de la imagen de contenido.


In [3]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [4]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

La función realiza una serie de transformaciones en la imagen para prepararla para ser procesada por la red neuronal VGG19:

- "load_img" carga la imagen del image_path dado y la redimensiona al tamaño objetivo (img_nrows x img_ncols).

- "img_to_array" convierte la imagen cargada en un array NumPy. Esto es necesario porque las redes neuronales no pueden trabajar directamente con imágenes, necesitan arrays numéricos.

- "expand_dims" agrega una dimensión extra al principio del array de la imagen. Esto se hace porque los modelos de redes neuronales convolucionales esperan un batch de imágenes como entrada, incluso si es un solo ejemplo, por lo que se agrega la dimensión del batch.

- "preprocess_input" realiza un preprocesamiento específico para la red VGG19. La función convierte las imágenes de RGB a BGR, luego centra en cero cada canal de color con respecto al conjunto de datos de ImageNet, sin escalar. Esto es necesario porque la red VGG19 fue entrenada en el conjunto de datos de ImageNet, que utiliza este tipo de preprocesamiento.

Por lo tanto, estas dos últimas líneas son necesarias para asegurar que la imagen de entrada esté en el formato correcto y tenga las características correctas para ser procesada por la red VGG19.

In [5]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

La función deprocess_image realiza el proceso inverso al de la función preprocess_image. Convierte una imagen procesada por la red VGG19 de nuevo a su formato original para que pueda ser visualizada correctamente.

- "x.reshape" se utiliza para reorganizar el array x con el fin de tener las dimensiones correctas para una imagen RGB con tamaño (img_nrows, img_ncols).

- Luego se deshace la normalización realizada en el preprocesamiento:

- En tercer lugar se convierte la representación de color de 'BGR' (Blue, Green, Red) a 'RGB' (Red, Green, Blue) invirtiendo el orden de los canales de color

- Por ultimo se asegura que los valores de píxeles estén en el rango válido [0, 255] y los convierte a tipo de dato 'uint8' (entero sin signo de 8 bits):

La relación con el código anterior ("preprocess_image") es que "deprocess_image" se utiliza para revertir el preprocesamiento de las imágenes que han sido generadas o manipuladas por el modelo.


In [6]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [8]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [9]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [10]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])


Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

La matriz de Gram se define en términos de los mapas de características de una capa de la red. Es decir, dada una matriz de características de una capa convolucional, la matriz de Gram se calcula como el producto punto de las distintas filas de la matriz de caracteristicas. La idea detrás de la matriz de Gram es capturar las correlaciones estadísticas entre las distintas características de la capa. 

En el contexto de la transferencia de estilo, la matriz de Gram se utiliza para cuantificar el estilo de una imagen. Al comparar las matrices de Gram de la imagen de estilo de referencia y la imagen generada, se puede evaluar cómo de cerca la imagen generada se asemeja al estilo de la imagen de referencia.

El proceso general de transferencia de estilo implica minimizar las diferencias entre las matrices de Gram de las capas de estilo de la imagen de referencia y la imagen generada, mientras se mantiene la estructura visual de la imagen de contenido. Esto se logra ajustando los pesos en la función de pérdida durante el proceso de optimización. Siendo que la función de pérdida de estilo se calcula comparando las matrices de Gram de la imagen generada y de la imagen de estilo.

- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram se calcula a partir de los mapas de características de la imagen de entrada. La función "K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))" aplana los mapas de características y los reorganiza para que puedan calcularse correctamente las correlaciones entre las características. 

Se hace para reorganizar los datos de entrada de manera que se puedan calcular correctamente las características de la matriz de Gram.

In [11]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

- style_loss: mide la diferencia de estilo entre la imagen de estilo y la imagen generada. Utiliza la matriz de Gram de ambas imágenes y calcula la suma de las diferencias cuadradas entre estas matrices. La pérdida de estilo intenta minimizar esta diferencia, lo que significa que intenta hacer que la imagen generada tenga un estilo similar al de la imagen de estilo.

- content_loss: mide la diferencia de contenido entre la imagen base (la imagen a la que se le quiere transferir el estilo) y la imagen generada. Calcula la suma de las diferencias cuadradas entre estas dos imágenes. La pérdida de contenido intenta minimizar esta diferencia, lo que significa que intenta hacer que la imagen generada tenga un contenido similar al de la imagen base.

- total_variation_loss: se utiliza para suavizar la imagen generada. Calcula la suma de las diferencias cuadradas entre los píxeles adyacentes en la imagen generada. Esta pérdida intenta minimizar estas diferencias, lo que significa que intenta hacer que la imagen generada sea más suave y menos ruidosa.

In [12]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [13]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [14]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [15]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [16]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

Celda 1: Define una función eval_loss_and_grads que toma una imagen (en forma de un array 1D), la remodela a su forma original, y luego calcula el valor de la pérdida y los gradientes para esa imagen utilizando la función f_outputs. Esta función es necesaria porque el optimizador que se utiliza más adelante necesita una forma de calcular tanto la pérdida como los gradientes para poder optimizar la imagen.

Celda 2: Define una clase Evaluator que se utiliza para calcular y almacenar la pérdida y los gradientes. Esto se hace porque el optimizador fmin_l_bfgs_b requiere que la pérdida y los gradientes se calculen por separado, pero calcularlos por separado sería ineficiente. Por lo tanto, la clase Evaluator calcula tanto la pérdida como los gradientes en un solo paso y luego los almacena para su uso posterior.

Celda 3: Crea una instancia de la clase Evaluator y luego utiliza el optimizador fmin_l_bfgs_b para minimizar la pérdida de estilo neuronal. Esto se hace iterativamente, y después de cada iteración, la imagen generada se guarda.

La función fmin_l_bfgs_b es una implementación del algoritmo L-BFGS-B que se utiliza para minimizar una función. En este caso, la función que se minimiza es la pérdida de estilo neuronal. La función fmin_l_bfgs_b toma como entrada una función de pérdida, una imagen inicial, y una función que calcula los gradientes.

La implementación del paper utiliza la red VGG-19, una red neuronal convolucional pre-entrenada, para extraer las representaciones de contenido y estilo de las imágenes.

Optimización: El paper no especifica qué algoritmo de optimización se utiliza para minimizar la función de pérdida. Aca se utiliza el algoritmo L-BFGS-B para minimizar la función de pérdida.
Normalización: En el paper, no se menciona ninguna normalización específica de las imágenes. Aca las imágenes se normalizan antes de ser procesadas por la red.
Iteraciones: En el paper, no se especifica el número de iteraciones utilizadas para la optimización. Aca se realiza un número fijo de iteraciones para la optimización.

En cuanto a las alternativas a fmin_l_bfgs_b, existen otros algoritmos de optimización que podrían utilizarse, como el descenso de gradiente estocástico (SGD), Adam, o RMSprop. Sin embargo, fmin_l_bfgs_b es un buen optimizador para este tipo de problemas debido a su eficiencia y precisión.


In [17]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [18]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [20]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('Ejercicio_9_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 57918580000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_0.png
Iteration 0 completed in 110s
Start of iteration 1
Current loss value: 23124046000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_1.png
Iteration 1 completed in 115s
Start of iteration 2
Current loss value: 15553452000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_2.png
Iteration 2 completed in 134s
Start of iteration 3
Current loss value: 11679978000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_3.png
Iteration 3 completed in 127s
Start of iteration 4
Current loss value: 9535989000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_4.png
Iteration 4 completed in 120s
Start of iteration 5
Current loss value: 8433246700.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_5.png
Iteration 5 comple

Current loss value: 3494675000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_48.png
Iteration 48 completed in 120s
Start of iteration 49
Current loss value: 3479739000.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_49.png
Iteration 49 completed in 123s
Start of iteration 50
Current loss value: 3466101800.0
Image saved as C:\Users\WonderBrands\Desktop\ITBA\TP final\output\Ejercicio_9_50.png
Iteration 50 completed in 117s
Start of iteration 51


KeyboardInterrupt: 

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:
Se realizaron tres combinaciones diferentes, para cada una se adjuntan 5 imagenes de titulo por ejemplo "Ejercicio_8_1(total_variation_weight = 1, style_weight = 0.1, content_weight = 5)". Es decir, se indica la combinacion y el numero de iteracion de la imagen. 

Primer combinación:
- total_variation_weight = 1
- style_weight = 0.1
- content_weight = 5 

Con esta configuración, la imagen generada se parecerá más a la imagen de contenido original.

Segunda combinación:
- total_variation_weight = 1
- style_weight = 1
- content_weight = 0.5 

Con esta configuración, la imagen generada adoptará más del estilo de la imagen de referencia.

Tercera Combinación:
- total_variation_weight = 5
- style_weight = 0.1
- content_weight = 1 

Con esta configuración, la imagen generada será más suave y tendrá menos ruido.


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Para este ejercicio se debe reiniciar el kernel y correr la celda que carga las imagenes del ejercicio (se adjuntan las imagenes utilizadas para que se pueda reproducir el mismo), como tambien se adjuntan 50 iteraciones del ejercicio.
